In [ ]:
''''In tis main file we do an initial explortation, we clean the data, and perfoirm EDA and vizualisation'''

import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import initial_exploration as explo
import data_cleaning as cl

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

In [ ]:
# Load datasets:
cash_requests = pd.read_csv('project_dataset/extract - cash request - data analyst.csv')
fees = pd.read_csv('project_dataset/extract - fees - data analyst - .csv')

In [ ]:
cash_requests.head()

In [ ]:
# Inital exploration:
explo.check(cash_requests)

In [ ]:
cl.convert_dates(cash_requests, cl.cash_request_date_columns)
cash_requests.head()

In [ ]:
fees.head()

In [ ]:
explo.check(fees)

In [ ]:
cl.convert_dates(fees, cl.fees_data_date_columns)
fees.head()

In [ ]:
data_df = cl.merge_df(cash_requests, fees, 'outer','id', 'cash_request_id')
data_df.head()

In [ ]:
explo.check(data_df)

In [ ]:
data_df = cl.remove_nan(data_df, "id_x")
explo.check_null(data_df)

In [ ]:
frames = cl.selecting_data_types(data_df)
num = frames[1]
cat = frames[0]

In [ ]:
num.head()

In [ ]:
cat.head()

In [ ]:

# then we can selecet numerical and categorical columns , and divide the columns between us and ecah work in a set of columns performing an 
# univariate analysis (frequency plots, box plots, doughnut, histograms   or any other you concider)
#For numercial plots also create report in mean , meadian, std, and arrive to conclusions 